In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [2]:
df_imo_search = pd.read_csv('./data/vessel_particular/imo_search_result.csv')

In [3]:
df_imo_search.shape

(382, 32)

In [4]:
df_imo_search.columns.values

array(['id', 'imo', 'mmsi', 'name', 'type', 'year_build',
       'operating_status', 'gt', 'dwt', 'length', 'breadth', 'draft',
       'engine_type', 'kw', 'flag_id', 'builder_id', 'class_society_id',
       'home_port_id', 'updated', 'verified', 'builder', 'class_society',
       'owner_id', 'owner_name', 'manager_id', 'manager_name',
       'former_names', 'seafarer_count', 'vacancy_count', 'home_port',
       'engine_model_id', 'engine_model_name'], dtype=object)

In [5]:
required_col = ['imo', 'type','dwt', 'length', 'breadth', 'manager_name', 'owner_name']
df_vessel_particular = df_imo_search[required_col]

In [6]:
df_vessel_particular.isnull().sum()

imo              0
type            26
dwt             26
length          26
breadth         26
manager_name    68
owner_name      67
dtype: int64

In [7]:
df_vessel_particular[df_vessel_particular['type'].isnull()]

,imo,type,dwt,length,breadth,manager_name,owner_name
203,9741449,NaN,NaN,NaN,NaN,NaN,NaN
209,9814600,NaN,NaN,NaN,NaN,NaN,NaN
216,9814612,NaN,NaN,NaN,NaN,NaN,NaN
223,9857169,NaN,NaN,NaN,NaN,NaN,NaN
224,9741384,NaN,NaN,NaN,NaN,NaN,NaN
239,9857183,NaN,NaN,NaN,NaN,NaN,NaN
244,9839923,NaN,NaN,NaN,NaN,NaN,NaN
251,9757852,NaN,NaN,NaN,NaN,NaN,NaN
260,9778105,NaN,NaN,NaN,NaN,NaN,NaN
267,9857157,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# some results actually contains NaN for all required values. So, we just drop them 
df_vessel_particular = df_vessel_particular[df_vessel_particular['type'].notnull()]

In [9]:
df_vessel_particular.isnull().sum()

imo              0
type             0
dwt              0
length           0
breadth          0
manager_name    42
owner_name      41
dtype: int64

In [10]:
df_vessel_particular[(df_vessel_particular['manager_name'].isnull()) & (df_vessel_particular['owner_name'].notnull())]

,imo,type,dwt,length,breadth,manager_name,owner_name
220,9728253,12.0,115343.0,299.0,48.0,NaN,"CAPITAL SHIPMANAGEMENT - ATHENS, GREECE"


In [11]:
df_owner_for_implant = df_vessel_particular[(df_vessel_particular['manager_name'].isnull()) & (df_vessel_particular['owner_name'].notnull())]

In [12]:
# We'll use owner_name to implant missing manager_name
for index, row in df_owner_for_implant.iterrows():
    print(f"IMO: {row.imo}, Owner Name: {row.owner_name}")
    #df_vessel_particular[df_vessel_particular['imo'] == row['imo']].loc[:, 'manager_name'] = row['owner_name']
    df_vessel_particular.at[index, 'manager_name'] = row['owner_name']

IMO: 9728253, Owner Name: CAPITAL SHIPMANAGEMENT - ATHENS, GREECE


In [13]:
df_vessel_particular[(df_vessel_particular['manager_name'].isnull()) & (df_vessel_particular['owner_name'].notnull())]

,imo,type,dwt,length,breadth,manager_name,owner_name


In [14]:
df_vessel_particular.isnull().sum()

imo              0
type             0
dwt              0
length           0
breadth          0
manager_name    41
owner_name      41
dtype: int64

In [15]:
# Format Vessel Operator by removing anything after - 
def format_vessel_operator(x):
    if not pd.isna(x):
        return x.split('-')[0].strip()
    else:
        return None

In [16]:
df_vessel_particular['manager_name'] = df_vessel_particular['manager_name'].apply(lambda x: format_vessel_operator(x))


In [17]:
df_vessel_particular.isnull().sum()

imo              0
type             0
dwt              0
length           0
breadth          0
manager_name    41
owner_name      41
dtype: int64

In [18]:
#df_vessel_particular.to_csv('./assets/vessel_particular/vessel_particular_temp.csv', index=False)

In [19]:
# futher map the vessel operator cleaned manaully 
df_mapping = pd.read_csv('./data/vessel_particular/vessel_operator_mapping.csv')

In [20]:
df_mapping.head(3)

,imo,original_operator,mapped_operator
0,7617905,HORIZON LINES,Pasha Hawaii Holdings LLC
1,7729459,HORIZON LINES,Pasha Hawaii Holdings LLC
2,7729461,HORIZON LINES,Pasha Hawaii Holdings LLC


In [21]:
df_mapping['mapped_operator'] = df_mapping['mapped_operator'].str.upper()

In [22]:
df_vessel_particular.shape

(356, 7)

In [23]:
df_vessel_particular = pd.merge(df_vessel_particular, df_mapping, how='left', left_on=['imo'], right_on=['imo']) 

In [24]:
df_vessel_particular.shape

(356, 9)

In [25]:
df_vessel_particular['manager_name'] = df_vessel_particular['mapped_operator']

In [26]:
df_vessel_particular = df_vessel_particular.drop(columns=['original_operator', 'mapped_operator'])

In [27]:
df_vessel_particular.isnull().sum()

imo              0
type             0
dwt              0
length           0
breadth          0
manager_name     1
owner_name      41
dtype: int64

In [28]:
df_vessel_particular = df_vessel_particular[df_vessel_particular['manager_name'].notnull()]

In [29]:
df_vessel_particular.shape

(355, 7)

#### Vessel Type Mapping

In [30]:
df_vessel_particular['type'].unique()

array([12., 21., 14.,  9., 32., 15.])

In [31]:
vessel_type_mapping = {
    12:'Container ship',
    21:'RO-RO',
    9:'Bulk carrier',
    14:'General cargo vessel',
    32:'LPG carrier',
    15:'Heavy lift vessel'
}

In [32]:
df_vessel_particular['vessel_type'] = df_vessel_particular['type'].map(vessel_type_mapping)

In [33]:
df_vessel_particular.head()

,imo,type,dwt,length,breadth,manager_name,owner_name,vessel_type
0,9215880,12.0,34677.0,210.0,30.0,MAERSK,"BSM BERMUDA - HAMILTON, BERMUDA",Container ship
1,7907984,12.0,30825.0,262.0,32.0,MATSON,"MATSON NAVIGATION - SAN FRANCISCO CA, United S...",Container ship
2,9619684,21.0,24750.0,210.0,32.0,PASHA HAWAII HOLDINGS LLC,"PASHA HAWAII SHIPPING - HONOLULU HI, United St...",RO-RO
3,9278105,12.0,50829.0,260.0,32.0,ZIM LINES,NaN,Container ship
4,9345427,12.0,109950.0,349.0,45.0,COSCO SHIPPING LINES,"COSCON - SHANGHAI, CHINA",Container ship


In [34]:
# drop unused columns 
df_vessel_particular = df_vessel_particular.drop(columns=['type', 'owner_name'])

In [35]:
df_vessel_particular = df_vessel_particular.rename(
    columns={
        'dwt':'vessel_dwt',
        'length':'vessel_length',
        'breadth':'vessel_width',
        'manager_name':'vessel_operator'
    }
)

In [36]:
# add 'IMO' to the IMO value so it can join with AIS data 
df_vessel_particular['imo'] = 'IMO' + df_vessel_particular['imo'].astype(str)

In [37]:
df_vessel_particular.head(3)

,imo,vessel_dwt,vessel_length,vessel_width,vessel_operator,vessel_type
0,IMO9215880,34677.0,210.0,30.0,MAERSK,Container ship
1,IMO7907984,30825.0,262.0,32.0,MATSON,Container ship
2,IMO9619684,24750.0,210.0,32.0,PASHA HAWAII HOLDINGS LLC,RO-RO


#### Merge with AIS data 

In [38]:
df_ais_final = pd.read_csv('./data/final_output/ais_final.csv')

In [39]:
df_ais_final.shape

(1779, 11)

In [40]:
df_vessel_particular.shape

(355, 6)

In [41]:
df_merged = pd.merge(df_ais_final, df_vessel_particular, how='left', left_on=['imo'], right_on=['imo'])

In [42]:
df_merged.isnull().sum()

imo                                   0
vessel_name                           0
time_seen                             0
target_terminal                       0
dwell_in_hr                           0
avg_dwell_at_target_terminal          0
num_of_vessel_at_target_terminal      0
num_of_vessel_in_port                 0
weekday                               0
hour_of_day                           0
is_holiday                            0
vessel_dwt                          145
vessel_length                       145
vessel_width                        145
vessel_operator                     145
vessel_type                         145
dtype: int64

In [43]:
df_merged.shape

(1779, 16)

In [44]:
# we will drop the 173 records which don't have vessel particular information 
df_merged = df_merged.dropna()

In [45]:
df_merged.shape

(1634, 16)

In [46]:
df_merged.head(3)

,imo,vessel_name,time_seen,target_terminal,dwell_in_hr,avg_dwell_at_target_terminal,num_of_vessel_at_target_terminal,num_of_vessel_in_port,weekday,hour_of_day,is_holiday,vessel_dwt,vessel_length,vessel_width,vessel_operator,vessel_type
0,IMO9215880,MAERSK NEWHAVEN,2020-02-04 10:15:50,PierT,75.7,25.5,1.0,50.0,1,10,False,34677.0,210.0,30.0,MAERSK,Container ship
1,IMO9215880,MAERSK NEWHAVEN,2020-03-22 10:16:06,PierT,53.2,46.4,2.0,53.0,6,10,False,34677.0,210.0,30.0,MAERSK,Container ship
2,IMO7907984,MANOA,2020-02-05 00:23:12,PierA,11.6,14.0,0.0,57.0,2,0,False,30825.0,262.0,32.0,MATSON,Container ship


In [47]:
df_merged = df_merged[
            ['imo',
            'vessel_name',
            'time_seen',
            'target_terminal',
            'avg_dwell_at_target_terminal',
            'num_of_vessel_at_target_terminal',
            'num_of_vessel_in_port',
            'weekday',
            'hour_of_day',
            'is_holiday',
            'vessel_operator',
            'vessel_type',
            'vessel_width',
            'vessel_length',
            'vessel_dwt',
            'dwell_in_hr']
        ]

In [48]:
df_merged.to_csv('./data/final_output/vessel_dwell_time.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6b18b33d-3a56-4f49-ad6e-71ecea9f0183' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>